In [1]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


**Import Question Bank**

In [2]:
with open("f1_question_bank.json", "r", encoding="utf-8") as f:
    QA_KB = json.load(f)

In [3]:
docs = [(item["q"] + " " + item["a"]) for item in QA_KB]
questions = docs

In [4]:
docs

['Who won the most races in the 2022 season? Max Verstappen won the most races in the 2022 Formula 1 season.',
 'Which constructor had the highest total points in 2022? Red Bull Racing had the highest total points in 2022.',
 'Which driver had the most podium finishes in 2022? Max Verstappen had the most podium finishes in 2022.',
 'Is there a correlation between starting grid position and finishing position? Yes. Drivers who start closer to the front generally finish better, but strategy/incidents can change outcomes.',
 'Did pit stop duration affect final race position? Often yes—slow stops can cost track position, especially where overtaking is difficult.',
 'Which circuit hosted the most races historically? Monza is among the circuits that have hosted the most F1 races historically.',
 'What is the average finishing position per driver? Average finishing position is the mean of a driver’s finishing positions across races (lower is better).',
 'Which driver had the most DNFs? It dep

**Key Words**

In [5]:
keywords = {
    "dnf": ["dnf", "did not finish", "retire", "retired", "retirement", "crash", "mechanical"],
    "podium": ["podium", "top 3", "p1", "p2", "p3"],
    "wins": ["win", "wins", "won", "victory", "victories"],
    "points": ["points", "standings", "championship"],
    "pit": ["pit", "pitstop", "pit stop", "duration", "undercut", "overcut"],
    "grid": ["grid", "qualifying", "start", "started"],
    "lap": ["lap", "lap time", "laptime", "milliseconds"],
    "circuit": ["circuit", "track", "monza", "spa", "silverstone", "location"],
    "constructor": ["constructor", "team", "red bull", "mercedes", "ferrari"]
}

**Clean text**

In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

**TF-IDF**

In [7]:
q = [clean_text(item["q"]) for item in QA_KB]
df = pd.DataFrame(q,columns=["questions"])
df.head()

,questions
0,who won the most races in the 2022 season
1,which constructor had the highest total points...
2,which driver had the most podium finishes in 2022
3,is there a correlation between starting grid p...
4,did pit stop duration affect final race position


In [8]:
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["questions"])
feature_names = vectorizer.get_feature_names_out()
results = []

for i in range(X.shape[0]):
    row = X[i].toarray().flatten()
    word_scores = {
        feature_names[j]: round(row[j], 3)
        for j in range(len(row)) if row[j] > 0
    }
    results.append(word_scores)

df["TFIDF"] = results

df.head()

,questions,TFIDF
0,who won the most races in the 2022 season,"{'2022': 0.452, 'races': 0.494, 'season': 0.49..."
1,which constructor had the highest total points...,"{'2022': 0.377, 'constructor': 0.463, 'highest..."
2,which driver had the most podium finishes in 2022,"{'2022': 0.461, 'driver': 0.461, 'finishes': 0..."
3,is there a correlation between starting grid p...,"{'correlation': 0.394, 'finishing': 0.351, 'gr..."
4,did pit stop duration affect final race position,"{'affect': 0.361, 'did': 0.361, 'duration': 0...."


In [9]:
#vectorizer = TfidfVectorizer(stop_words="english",norm=None)
#X = vectorizer.fit_transform(df["questions"])
def dotproduct(query):
    query_vec = vectorizer.transform([query])
    query_array = query_vec.toarray()[0]

    dot_products = np.dot(X, query_vec.T).toarray().flatten()
    doc_magnitudes = np.linalg.norm(X.toarray(), axis=1)
    query_magnitude = np.linalg.norm(query_vec.toarray())
    cosine_scores = dot_products / (doc_magnitudes * query_magnitude + 1e-10)
    out = pd.DataFrame({
        "Query": query,
        "questions": df["questions"].values,
        "Dot_Product": dot_products,
        "Doc Magnitude": doc_magnitudes,
        "Query_Magnitude": np.full(len(df), query_magnitude),
        "Cosine_Similarity": cosine_scores
    })

    return out

In [10]:
dotproduct("most races 2022")

,Query,questions,Dot_Product,Doc Magnitude,Query_Magnitude,Cosine_Similarity
0,most races 2022,who won the most races in the 2022 season,0.669573,1.0,1.0,0.669573
1,most races 2022,which constructor had the highest total points...,0.254478,1.0,1.0,0.254478
2,most races 2022,which driver had the most podium finishes in 2022,0.311006,1.0,1.0,0.311006
3,most races 2022,is there a correlation between starting grid p...,0.000000,1.0,1.0,0.000000
4,most races 2022,did pit stop duration affect final race position,0.000000,1.0,1.0,0.000000
5,most races 2022,which circuit hosted the most races historically,0.337879,1.0,1.0,0.337879
6,most races 2022,what is the average finishing position per driver,0.000000,1.0,1.0,0.000000
7,most races 2022,which driver had the most dnfs,0.000000,1.0,1.0,0.000000
8,most races 2022,which circuits produce the most overtakes,0.000000,1.0,1.0,0.000000
9,most races 2022,can we predict a podium finish,0.000000,1.0,1.0,0.000000


**Vectorization**

In [11]:
vectorizer = TfidfVectorizer(preprocessor=clean_text, ngram_range=(1,2))
question_vectors = vectorizer.fit_transform(questions)

**Retrieval Funciton**

In [12]:
def get_answer(user_query, threshold=0.05):
    query_vec = vectorizer.transform([user_query])
    similarities = cosine_similarity(query_vec, question_vectors).flatten()

    best_index = similarities.argmax()
    best_score = similarities[best_index]

    if best_score < threshold:
        return "I don't have an answer for that yet. Try asking about wins, podiums, DNFs, pit stops, or grid positions."

    return QA_KB[best_index]["a"]

In [13]:
def chat():
    print("🏎️ F1 FAQ Chatbot (type 'quit' to exit)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Bot: Goodbye!")
            break
        response = get_answer(user_input)
        print("Bot:", response)

**Chat Box**

In [14]:
if __name__ == "__main__":
    chat()

🏎️ F1 FAQ Chatbot (type 'quit' to exit)



You:  max verstappen


Bot: Max Verstappen drives for Red Bull Racing in the 2026 season.



You:  who is the current winner


Bot: Average finishing position is the mean of a driver’s finishing positions across races (lower is better).


...

Bot: Average finishing position is the mean of a driver’s finishing positions across races (lower is better).



You:  successful


Bot: I don't have an answer for that yet. Try asking about wins, podiums, DNFs, pit stops, or grid positions.



You:  quit


Bot: Goodbye!
